<a href="https://colab.research.google.com/github/antonysama/exploratory/blob/master/Petrinex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
#Install pkgs
install.packages(c('janitor', dependencies = TRUE, repos = "https://cran.rstudio.com/"))
x<-c('dplyr', 'magrittr','skimr','tidyr','janitor')
lapply(x, require, character.only = TRUE)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“packages ‘TRUE’, ‘https://cran.rstudio.com/’ are not available for this version of R

Versions of these packages for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”


[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

In [62]:
#Read csv
data <- read.csv("data.CSV", header = TRUE)
df <- data[, 1:24]  # Assuming columns A to X are indices 1 to 24
head(df,2)

,ProductionMonth,OperatorBAID,OperatorName,ReportingFacilityID,ReportingFacilityProvinceState,ReportingFacilityType,ReportingFacilityIdentifier,ReportingFacilityName,ReportingFacilitySubType,ReportingFacilitySubTypeDesc,⋯,FacilityRange,FacilityMeridian,SubmissionDate,ActivityID,ProductID,FromToID,FromToIDProvinceState,FromToIDType,FromToIDIdentifier,Volume
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<chr>,⋯,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2023-04,0007,IMPERIAL OIL RESOURCES LIMITED,ABBT0051211,AB,BT,51211,IMPERIAL MASKWA BATTERY 10-12,344,IN-SITU OIL SANDS,⋯,4,4,2023-05-19,,,,,,,
2,2023-04,0007,IMPERIAL OIL RESOURCES LIMITED,ABBT0051211,AB,BT,51211,IMPERIAL MASKWA BATTERY 10-12,344,IN-SITU OIL SANDS,⋯,4,4,2023-05-19,,,,,,,


In [63]:
#Inpect data quality & types
skim_without_charts(df)

── Data Summary ────────────────────────
                           Values
Name                       df    
Number of rows             556221
Number of columns          24    
_______________________          
Column type frequency:           
  character                17    
  numeric                  7     
________________________         
Group variables            None  

── Variable type: character ────────────────────────────────────────────────────
   skim_variable                  n_missing complete_rate min max  empty
 1 ProductionMonth                        0             1   7   7      0
 2 OperatorBAID                           0             1   4   4      0
 3 OperatorName                           0             1   9  97      0
 4 ReportingFacilityID                    0             1  11  11      0
 5 ReportingFacilityProvinceState         0             1   2   2      0
 6 ReportingFacilityType                  0             1   2   2      0
 7 ReportingFacilityName  

ERROR: ignored

In [64]:
# Convert 'Volume' to numeric, ReportingFacilitySubType to integer
df$Volume <- as.numeric(df$Volume)
df$ReportingFacilitySubType <- as.integer(df$ReportingFacilitySubType)
# Verify the data type of 'Volume'
str(df$Volume) #  class(df$Volume)
str(df$ReportingFacilitySubType) #  class(df$ReportingFacilitySubType)


Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


 num [1:556221] NA NA NA NA NA NA NA NA NA NA ...
 int [1:556221] 344 344 344 344 344 344 344 344 344 344 ...


The below 5 are the operations to convert to numbers we want

In [66]:
(oil<-sum((df$Volume * 6.2925 / 30)[df$ActivityID =="PROD" & df$ProductID == "OIL" & (df$ReportingFacilitySubType < 331 | df$ReportingFacilitySubType > 349) & !is.na(df$Volume)]))

[1] 530845.6

In [67]:
(cond<-sum((df$Volume * 6.2925 / 30)[df$ActivityID =="PROD" & df$ProductID == "COND" & !is.na(df$Volume)]))

[1] 67891.17

In [68]:
(thermal<-sum((df$Volume * 6.29234 / 30)[df$ActivityID =="PROD" & df$ProductID == "OIL" & df$ReportingFacilitySubType >= 331 & df$ReportingFacilitySubType <= 349 & !is.na(df$Volume)]))

[1] 1664531

In [69]:
(c5<-sum((df$Volume )[df$ActivityID =="PROC" & (df$ProductID == "C5-MX" | df$ProductID == "C5-SP")&!is.na(df$Volume)]))


[1] 1326286

In [71]:
(gas<-sum((df$Volume )[df$ActivityID =="PROD" & df$ProductID == "GAS" &!is.na(df$Volume)]))

[1] 11173237

In [72]:
#Print an array of results
c (oil, cond, thermal, c5, gas)
#oil-bpd, cond-bpd, thermal-bpd, c5-m3, gas-m3

[1]   530845.60    67891.17  1664530.72  1326285.60 11173237.40